## This notebook does slew simulations, and check all aos components (M1M3, M2, hexapods) behavior during the slew-and-track process

This is expected to work both for SUMMIT and NCSA

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

Patching auth into notebook.base.handlers.IPythonHandler(notebook.base.handlers.AuthenticatedHandler) -> IPythonHandler(jupyterhub.singleuser.mixins.HubAuthenticatedHandler, notebook.base.handlers.AuthenticatedHandler)


'andes06.cp.lsst.org'

In [3]:
from lsst.ts import salobj
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import logging

from lsst.ts.idl.enums import MTHexapod

from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u

from aosTools import *

plt.jet();

Update leap second table
current_tai uses the system TAI clock


<Figure size 432x288 with 0 Axes>

In [4]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [5]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

file:///home/hdrass/WORK/ts_ddsconfig/config/ospl-shmem.xml
summit
0


In [6]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [7]:
from lsst_efd_client import EfdClient

if summit:
    client = EfdClient('summit_efd')
else:
    client = EfdClient('ncsa_teststand_efd')
csc_index = 1

In [8]:
#index is an integter which helps avoid multple users starting same controller
script = salobj.Controller("Script", index=7)

Read historical data in 0.00 sec


In [9]:
mtcs = MTCS(script.domain)
mtcs.set_rem_loglevel(40)

In [10]:
await mtcs.start_task

elevation DDS read queue is full (100 elements); data may be lost
cameraCableWrap DDS read queue is full (100 elements); data may be lost
azimuth DDS read queue is full (100 elements); data may be lost
Read historical data in 0.15 sec
Read historical data in 0.20 sec
powerSupplyData DDS read queue is full (100 elements); data may be lost
timeAndDate DDS read queue is full (100 elements); data may be lost
zenithAngle DDS read queue is full (100 elements); data may be lost
electrical DDS read queue is filling: 86 of 100 elements


[None, None, None, None, None, None, None, None, None, None]

electrical DDS read queue is full (100 elements); data may be lost
rotation DDS read queue is full (100 elements); data may be lost
pidData DDS read queue is full (100 elements); data may be lost
mountStatus DDS read queue is full (100 elements); data may be lost
temperature DDS read queue is full (100 elements); data may be lost


In [11]:
mtcs.components_attr

application DDS read queue is filling: 89 of 100 elements
motors DDS read queue is full (100 elements); data may be lost
application DDS read queue is full (100 elements); data may be lost
inclinometerData DDS read queue is full (100 elements); data may be lost
mountPosition DDS read queue is full (100 elements); data may be lost
tangentForce DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is filling: 90 of 100 elements
electrical DDS read queue is full (100 elements); data may be lost
actuators DDS read queue is full (100 elements); data may be lost
imsData DDS read queue is full (100 elements); data may be lost
tangentEncoderPositions DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost
ccwFollowingError DDS read queue is full (100 elements); data may be lost
tangentActuatorSteps DDS read queue is full (100 elements); data may be lost
hardpointMonitorData DDS read queue is 

['mtmount',
 'mtptg',
 'mtaos',
 'mtm1m3',
 'mtm2',
 'mthexapod_1',
 'mthexapod_2',
 'mtrotator',
 'mtdome',
 'mtdometrajectory']

In [12]:
ptg = mtcs.rem.mtptg
mount = mtcs.rem.mtmount
rot = mtcs.rem.mtrotator
camhex = mtcs.rem.mthexapod_1
m2hex = mtcs.rem.mthexapod_2
m1m3 = mtcs.rem.mtm1m3
m2 = mtcs.rem.mtm2
#aos = mtcs.rem.mtaos

### Get Ptg ready

In [ ]:
await ptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await ptg.cmd_start.set_start()

In [ ]:
await salobj.set_summary_state(ptg, salobj.State.ENABLED)

### Get Mount ready

In [ ]:
await mount.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sim_evt = await mount.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
a = await mount.evt_softwareVersions.aget(timeout=5)
print('simulation mode? ', a.cscVersion, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
a = await mount.evt_cameraCableWrapFollowing.aget()
print(a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await salobj.set_summary_state(mount, salobj.State.OFFLINE)

In [ ]:
await mount.cmd_start.set_start()

In [50]:
await salobj.set_summary_state(mount, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [ ]:
await showSlewError(ptg, mount, rot)

### Get Rotator ready

In [ ]:
await checkSlewCompStates(ptg, mount, rot)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
a = await mount.tel_elevation.next(flush=True, timeout=5)
print("mount elevation Angle = ", a.actualPosition)
a = await mount.tel_azimuth.next(flush=True, timeout=5)
print("mount azimuth angle = ", a.actualPosition)
a = await mount.tel_cameraCableWrap.next(flush=True, timeout=5)
print("CCW angle = ", a.actualPosition, " Needs to be within 2.2 deg of rotator angle ")
b = await rot.tel_rotation.next(flush=True, timeout=5)
print("rot angle = ", b.actualPosition, "   diff = ", (b.actualPosition - a.actualPosition))

In [ ]:
a = await rot.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
#if you get the "CCW telemetry too old", check the below. The CSC checks timestamp, not private_sndStamp
a=await mount.tel_cameraCableWrap.next(flush=True, timeout=5)
print(pd.to_datetime(a.private_sndStamp, unit='s'), a.actualPosition)
print(pd.to_datetime(a.timestamp, unit='s'), a.actualPosition)
a=await rot.tel_rotation.next(flush=True, timeout=5)
print(pd.to_datetime(a.private_sndStamp, unit='s'), a.actualPosition)
print(pd.to_datetime(a.timestamp, unit='s'), a.actualPosition)

In [ ]:
sim_evt = await rot.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
#a = await mount.evt_cameraCableWrapState.aget(timeout=5.)
#print(a.state)
a = await mount.evt_cameraCableWrapFollowing.aget(timeout=5.)
print(a.enabled)

In [ ]:
await mount.cmd_enableCameraCableWrapFollowing.set_start()

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
await rot.cmd_start.set_start()

In [ ]:
await salobj.set_summary_state(mount, salobj.State.DISABLED)
await salobj.set_summary_state(mount, salobj.State.ENABLED)
a = await mount.evt_cameraCableWrapFollowing.aget()
print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await rot.cmd_move.set_start(0)

### test the slew simulations without AOS components

In [ ]:
#comment and uncomment the below depending on which components are expected to be on
#by default, MTCS checks on every component (during the slew).
mtcs.check.mtaos = False
mtcs.check.mtm1m3 = False
mtcs.check.mtm2 = False
mtcs.check.mthexapod_1 = False
mtcs.check.mthexapod_2 = False
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False

In [ ]:
dec = -34.
for j in range(2):
    time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
    ra = time_and_date.lst + 0.5
    for i in range(2):
        print(ra, dec)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=0., rot_type=RotType.PhysicalSky)
        await asyncio.sleep(39.)
        ra -= 3.5/15.0

await mtcs.stop_tracking()

In [ ]:
a =await mount.evt_target.aget(timeout=5.)
print(a.elevation)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.slew_icrs(ra=ra, dec=dec, rot=0.0, rot_type=RotType.Physical)

In [ ]:
dec = -34.
for j in range(2):
    for i in range(2):
        
        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
        ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
        
        aa = await mount.tel_elevation.next(flush=True, timeout=5.)
        current_el = aa.actualPosition
        
        obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
        azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
        target_el = azel.alt.value
        
        while abs(target_el - current_el)>0.3:
            print('moving from elevation %.1f deg to %.1f deg'%(current_el, target_el), Time.now())
            await moveMountConstantV(mount, current_el, target_el)
        
            aa = await mount.tel_elevation.next(flush=True, timeout=5.)
            current_el = aa.actualPosition
        
            time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
            ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
    
            obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
            azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
            target_el = azel.alt.value
        
        print('start a slew, elevation diff = ', abs(target_el - current_el), Time.now())

        #b = await mount.tel_cameraCableWrap.next(flush=True, timeout=5) #CCW doesn't follow rotator!!!
        #await mtcs.slew_icrs(ra=ra, dec=dec, rot=b.actualPosition, rot_type=RotType.PhysicalSky)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=0.0, rot_type=RotType.Physical)
        await asyncio.sleep(39.)
        await mtcs.stop_tracking()

### Check that all components are ready

In [ ]:
await checkAOSCompStates(m1m3, m2, camhex, m2hex)

In [ ]:
await checkSlewCompStates(ptg, mount, rot)

### Get M1M3 ready (Mount telemetry mode)

In [51]:
#Move to zenith (so that we can start m1m3 with LUT in mount telemetry mode)
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)

In [52]:
await m1m3.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

In [ ]:
print('Re-enabling M1M3')
#await salobj.set_summary_state(m1m3, salobj.State.STANDBY) 
await salobj.set_summary_state(m1m3, salobj.State.ENABLED, settingsToApply = 'Default') #enable m1m3

In [ ]:
await readyM1M3(m1m3)

In [ ]:
await m1m3.cmd_abortRaiseM1M3.set_start(timeout=15.)

In [ ]:
a = await m1m3.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [53]:
await lowerM1M3(m1m3)

m1m3 state DetailedState.LOWERING 2021-07-20 17:58:16.603898880
m1m3 state DetailedState.PARKED 2021-07-20 18:00:43.783618048


### Get M2 ready

In [ ]:
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await checkAOSCompStates(m1m3,m2,camhex,m2hex)

In [ ]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE) 

In [ ]:
await salobj.set_summary_state(m2, salobj.State.STANDBY) 

In [ ]:
await salobj.set_summary_state(m2, salobj.State.DISABLED) 

In [ ]:
#remember to reset interlock
await salobj.set_summary_state(m2, salobj.State.ENABLED) 

In [ ]:
a = await m2.evt_errorCode.aget()
print(a.errorCode, a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
await m2.cmd_enterControl.set_start(timeout=.5)

In [ ]:
await m2.cmd_exitControl.set_start(timeout=.5)

In [ ]:
await m2.cmd_enable.set_start(timeout=5.)

In [ ]:
await m2.cmd_disable.set_start(timeout=5.)

In [ ]:
await m2.cmd_clearErrors.set_start(timeout=15.)

In [ ]:
await readyM2(m2)

### Get camHex ready

In [ ]:
target_evt = await mount.evt_target.aget(timeout=5.)
print("Mount target elevation = ", target_evt.elevation, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
print("Mount target azimuth   = ", target_evt.azimuth, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))
target_evt = await rot.evt_target.aget(timeout=5.)
print("Rotator target position = ", target_evt.position, "  @  ", pd.to_datetime(target_evt.private_sndStamp, unit='s'))

In [ ]:
#if any of the above doesn't work
await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=90)
await rot.cmd_move.set_start(position=0)

In [ ]:
a=await mount.tel_cameraCableWrap.next(flush=True, timeout=5)
print(pd.to_datetime(a.private_sndStamp, unit='s'), a.actualPosition)
print(pd.to_datetime(a.timestamp, unit='s'), a.actualPosition)
a=await camhex.tel_application.next(flush=True, timeout=5)
print(pd.to_datetime(a.private_sndStamp, unit='s'), a.position[0])
#print(pd.to_datetime(a.timestamp, unit='s'), a.position[0])

In [ ]:
await camhex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
sim_evt = await camhex.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

In [ ]:
await camhex.cmd_clearError.set_start()

In [ ]:
await salobj.set_summary_state(camhex, salobj.State.ENABLED, settingsToApply='default')

In [30]:
await readyHexaForAOS(camhex)

settingsVersion =  default.yaml:v0.7.4-0-g70e7a58 2021-07-20 16:13:41.833638400
pivot at (0, 0, -2758400) microns 
maxXY =  11400.0 microns, maxZ=  13100.0  microns
maxUV =  0.36 deg, maxW=  0.1  deg
compsensation mode enabled? True 2021-07-20 16:17:32.289535744
hexa in position? False 2021-07-20 17:34:20.636253440
hexa in position? True 2021-07-20 17:34:22.167686912
Current Hexapod position
     -1.07    -559.48     394.29  -0.015243   0.000005  -0.000003
hexa in position? False 2021-07-20 17:34:22.347900160
hexa in position? True 2021-07-20 17:34:23.879484416
Current Hexapod position
     -0.92    -559.93     293.69  -0.015255   0.000001  -0.000010
Uncompensated position
      0.00       0.00       0.00      0.000000   0.000000   0.000000    2021-07-20 17:34:22.216455168
Compensated position
     -0.91    -559.36     293.93      -0.015243   0.000000   0.000000    2021-07-20 17:34:22.216644096
Does the hexapod has enough inputs to do LUT compensation? (If the below times out, we do no

In [38]:
await printHexaUncompensatedAndCompensated(m2hex)

Uncompensated position
      0.00       0.00       0.00      0.000000   0.000000   0.000000    2021-07-20 17:45:47.753095680
Compensated position
     -2.15    -491.08     307.31      -0.005733   0.000000   0.000000    2021-07-20 17:45:47.753287424


In [39]:
await printHexaUncompensatedAndCompensated(camhex)

Uncompensated position
      0.00       0.00       0.00      0.000000   0.000000   0.000000    2021-07-20 17:45:47.755436544
Compensated position
     -0.91    -560.92     294.00      -0.015284   0.000000   0.000000    2021-07-20 17:45:47.755623936


In [ ]:
import lsst.ts.idl.enums.MTHexapod as enum_hex
a = await camhex.evt_controllerState.aget()
print(a.applicationStatus, pd.to_datetime(a.private_sndStamp, unit='s'))
enum_hex.ApplicationStatus(a.applicationStatus[0])

### Get m2Hex ready

In [ ]:
await salobj.set_summary_state(m2hex, salobj.State.ENABLED, settingsToApply='default') #works if it is offline

In [ ]:
sim_evt = await m2hex.evt_simulationMode.aget(timeout=5)
print('simulation mode? ', sim_evt.mode, pd.to_datetime(sim_evt.private_sndStamp, unit='s'))

In [ ]:
await readyHexaForAOS(m2hex)

In [ ]:
end = Time(datetime.now(), scale='tai')
start = end - timedelta(seconds=1000)
df = await client.select_time_series('lsst.sal.ESS.temperature8Ch', '*', start, end, csc_index)
if len(df)>0:
    fig, ax = plt.subplots(1,1, figsize=(15,4))
    for i in range(8):
        plt.plot(getattr(df, 'temperatureC%02d'%i))
    plt.grid()
else:
    print('No temperature data on the camera hexapod in the past 1000s.')

### Do a few slews

In [13]:
#comment and uncomment the below depending on which components are expected to be on
#by default, MTCS checks on every component (during the slew).
mtcs.check.mtaos = False
mtcs.check.mtm1m3 = True
mtcs.check.mtm2 = True
mtcs.check.mthexapod_1 = True
mtcs.check.mthexapod_2 = True
mtcs.check.mtdome = False
mtcs.check.mtdometrajectory = False

In [14]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
    
await salobj.set_summary_state(mount, salobj.State.DISABLED)
await asyncio.sleep(1.0)
await salobj.set_summary_state(mount, salobj.State.ENABLED)

mount.evt_logMessage.callback = printLogMessage
await mount.cmd_setLogLevel.set_start(level=10, timeout=5)

10: Read {'id': 1, 'timestamp': 1626799596.8100486, 'parameters': {'commander': 1, 'sequenceId': 23785, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799596.9604461, 'parameters': {'commander': 1, 'sequenceId': 23786, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.1103418, 'parameters': {'commander': 1, 'sequenceId': 23787, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.260606, 'parameters': {'commander': 1, 'sequenceId': 23788, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.4105654, 'parameters': {'commander': 1, 'sequenceId': 23789, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.5614, 'parameters': {'commander': 1, 'sequenceId': 23790, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.7105758, 'parameters': {'commander': 1, 'sequenceId': 23791, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799597.8605685, 'parameters': {'commander': 1, 'sequenceId': 23792, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799598.0

In [17]:
mount.evt_logMessage.callback = None

10: Read {'id': 1, 'timestamp': 1626799696.0687265, 'parameters': {'commander': 1, 'sequenceId': 24447, 'timeout': -1}}


In [15]:
a = await mount.evt_cameraCableWrapFollowing.aget()
print(a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

#a = await mount.tel_elevation.next(flush=True, timeout=5.)
#await mount.cmd_moveToTarget.set_start(azimuth=0, elevation=a.actualPosition-1)
#await asyncio.sleep(1.0)
#a = await mount.evt_cameraCableWrapFollowing.aget()
#print(a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))

10: Read {'id': 1, 'timestamp': 1626799609.411364, 'parameters': {'commander': 1, 'sequenceId': 23869, 'timeout': -1}}
1 2021-07-20 16:46:36.794967552
10: Read {'id': 1, 'timestamp': 1626799609.5616634, 'parameters': {'commander': 1, 'sequenceId': 23870, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799609.7114975, 'parameters': {'commander': 1, 'sequenceId': 23871, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799609.8616621, 'parameters': {'commander': 1, 'sequenceId': 23872, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799610.0114996, 'parameters': {'commander': 1, 'sequenceId': 23873, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799610.1620517, 'parameters': {'commander': 1, 'sequenceId': 23874, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799610.3115995, 'parameters': {'commander': 1, 'sequenceId': 23875, 'timeout': -1}}
10: Read {'id': 1, 'timestamp': 1626799610.4614668, 'parameters': {'commander': 1, 'sequenceId': 23876, 'timeout': -1}}
10: Read 

In [ ]:
a = await mount.evt_logMessage.aget()
print(a.message)

#### Do 4 slews, then stop tracking

In [18]:
await rot.cmd_move.set_start(position=0, timeout=30.)

In [19]:
rot.evt_inPosition.flush()
await rot.cmd_move.set_start(position=10., timeout = 30)
while True:
    state = await rot.evt_inPosition.next(flush=False, timeout=30)
    print('rot in position?', state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
    if (state.inPosition):
        break

rot in position? False 2021-07-20 16:48:42.069686528
rot in position? True 2021-07-20 16:48:48.419933184


In [23]:
dec = -34.
for j in range(2):
    for i in range(2):
        
        a = await mount.evt_cameraCableWrapFollowing.aget()
        print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
        if not a.enabled:
            break
        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
        ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
        
        aa = await mount.tel_elevation.next(flush=True, timeout=5.)
        current_el = aa.actualPosition
        
        obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
        azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
        target_el = azel.alt.value
        
        while abs(target_el - current_el)>0.3:
            print('moving from elevation %.1f deg to %.1f deg'%(current_el, target_el), Time.now())
            await moveMountConstantV(mount, current_el, target_el)
        
            aa = await mount.tel_elevation.next(flush=True, timeout=5.)
            current_el = aa.actualPosition
        
            time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
            ra = time_and_date.lst + 0.5 - 3.5/15.0 * i
    
            obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
            azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
            target_el = azel.alt.value
        
        rot.evt_inPosition.flush()
        await rot.cmd_move.set_start(position=10., timeout = 30)
        while True:
            try:
                state = await rot.evt_inPosition.next(flush=False, timeout=30)
            except asyncio.TimeoutError:
                print("No inPosition event seen. Continuing")    
                break
            print('rot in position?', state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
            if (state.inPosition):
                break
        
        print('start a slew, elevation diff = ', abs(target_el - current_el), Time.now())
        a = await mount.evt_cameraCableWrapFollowing.aget()
        print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
        if not a.enabled:
            break
        rotAngle = 10.0
        print('rot = ', rotAngle)
        #b = await mount.tel_cameraCableWrap.next(flush=True, timeout=5) #CCW doesn't follow rotator!!!
        # await mtcs.slew_icrs(ra=ra, dec=dec, rot=b.actualPosition, rot_type=RotType.PhysicalSky)
        await mtcs.slew_icrs(ra=ra, dec=dec, rot=rotAngle, rot_type=RotType.PhysicalSky)
        #await mtcs.slew_icrs(ra=ra, dec=dec, rot=0.0, rot_type=RotType.Physical)
        await asyncio.sleep(39.)
        await mtcs.stop_tracking()

CCW folowing?  1 2021-07-20 16:46:36.794967552
No inPosition event seen. Continuing
start a slew, elevation diff =  0.07696001442772626 2021-07-20 16:58:35.452298
CCW folowing?  1 2021-07-20 16:46:36.794967552
rot =  10.0


Got False
Got True
Rotator in position.
Waiting for telescope to settle.
Telescope in position.


CCW folowing?  1 2021-07-20 16:46:36.794967552
moving from elevation 82.7 deg to 84.8 deg 2021-07-20 16:59:42.334291
This will run for 98 seconds
rot in position? True 2021-07-20 17:02:24.687124736
start a slew, elevation diff =  0.07748745930851442 2021-07-20 17:01:47.695666
CCW folowing?  1 2021-07-20 16:46:36.794967552
rot =  10.0


Got False
Got True
Rotator in position.
Waiting for telescope to settle.
Telescope in position.


CCW folowing?  1 2021-07-20 16:46:36.794967552
moving from elevation 84.9 deg to 82.4 deg 2021-07-20 17:02:56.377883
This will run for 120 seconds
rot in position? True 2021-07-20 17:06:23.556926208
start a slew, elevation diff =  0.07255114417604602 2021-07-20 17:05:46.564718
CCW folowing?  1 2021-07-20 16:46:36.794967552
rot =  10.0


Got False
Got True
Rotator in position.
Waiting for telescope to settle.
Telescope in position.


CCW folowing?  1 2021-07-20 16:46:36.794967552
moving from elevation 82.6 deg to 84.8 deg 2021-07-20 17:06:53.081753
This will run for 102 seconds
rot in position? True 2021-07-20 17:10:00.224952064
start a slew, elevation diff =  0.09281566069735447 2021-07-20 17:09:23.233408
CCW folowing?  1 2021-07-20 16:46:36.794967552
rot =  10.0


Got False
Got True
Rotator in position.
Waiting for telescope to settle.
Telescope in position.


In [29]:
#Send the telescope to tracking
a = await mount.evt_cameraCableWrapFollowing.aget()
print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
#if not a.enabled:
#    break
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
ra = time_and_date.lst + 0.5 - 3.5/15.0 * i

aa = await mount.tel_elevation.next(flush=True, timeout=5.)
current_el = aa.actualPosition

obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
target_el = azel.alt.value

while abs(target_el - current_el)>0.3:
    print('moving from elevation %.1f deg to %.1f deg'%(current_el, target_el), Time.now())
    await moveMountConstantV(mount, current_el, target_el)

    aa = await mount.tel_elevation.next(flush=True, timeout=5.)
    current_el = aa.actualPosition

    time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.next(flush=True, timeout=5)
    ra = time_and_date.lst + 0.5 - 3.5/15.0 * i

    obs_time = salobj.astropy_time_from_tai_unix(salobj.current_tai() + 0.) #with 0s delay
    azel = mtcs.azel_from_radec(ra=ra, dec=dec, time=obs_time)
    target_el = azel.alt.value

rot.evt_inPosition.flush()
await rot.cmd_move.set_start(position=10., timeout = 30)
while True:
    try:
        state = await rot.evt_inPosition.next(flush=False, timeout=30)
    except asyncio.TimeoutError:
        print("No inPosition event seen. Continuing")    
        break
    print('rot in position?', state.inPosition, pd.to_datetime(state.private_sndStamp, unit='s'))
    if (state.inPosition):
        break

print('start a slew, elevation diff = ', abs(target_el - current_el), Time.now())
a = await mount.evt_cameraCableWrapFollowing.aget()
print('CCW folowing? ', a.enabled, pd.to_datetime(a.private_sndStamp, unit='s'))
#if not a.enabled:
    #break
rotAngle = 10.0
print('rot = ', rotAngle)
#b = await mount.tel_cameraCableWrap.next(flush=True, timeout=5) #CCW doesn't follow rotator!!!
# await mtcs.slew_icrs(ra=ra, dec=dec, rot=b.actualPosition, rot_type=RotType.PhysicalSky)
await mtcs.slew_icrs(ra=ra, dec=dec, rot=rotAngle, rot_type=RotType.PhysicalSky)
#await mtcs.slew_icrs(ra=ra, dec=dec, rot=0.0, rot_type=RotType.Physical)
await asyncio.sleep(39.)

CCW folowing?  1 2021-07-20 16:46:36.794967552
rot in position? True 2021-07-20 17:23:52.793615872
start a slew, elevation diff =  0.17188417604749873 2021-07-20 17:23:15.808775
CCW folowing?  1 2021-07-20 16:46:36.794967552
rot =  10.0


Got False
Waiting for telescope to settle.
Telescope in position.
Got True
Rotator in position.


In [40]:
await mtcs.stop_tracking()

In [24]:
await showSlewError(ptg, mount, rot)

Camera cable wrap telemetry is too old: dt=1861.51021027565; abs(dt) > 1 2021-07-15 16:04:09.869033216
CCW folowing?  1 2021-07-20 16:46:36.794967552


In [25]:
await checkSlewCompStates(ptg, mount, rot)

staring with: ptg state State.ENABLED 2021-07-20 16:38:09.801031168
staring with: mount state State.ENABLED 2021-07-20 16:46:36.796051456
staring with: rot state State.ENABLED 2021-07-20 16:37:52.965134080


In [ ]:
a = await rot.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))
a = await rot.evt_logMessage.aget()
print(a.message, pd.to_datetime(a.private_sndStamp, unit='s'))
import lsst.ts.idl.enums.MTRotator as enum_rot
a = await rot.evt_controllerState.aget()
print(a.applicationStatus, pd.to_datetime(a.private_sndStamp, unit='s'))
enum_rot.ApplicationStatus(a.applicationStatus)

In [ ]:
a = await camhex.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
import lsst.ts.idl.enums.MTHexapod as enum_hex
a = await camhex.evt_controllerState.aget()
print(a.applicationStatus, pd.to_datetime(a.private_sndStamp, unit='s'))
enum_hex.ApplicationStatus(a.applicationStatus[0])

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
await checkSlewCompStates(ptg, mount, rot)

In [ ]:
await salobj.set_summary_state(rot, state=salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(ptg, state=salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(mount, state=salobj.State.DISABLED)

In [ ]:
await salobj.set_summary_state(mount, state=salobj.State.ENABLED)

In [ ]:
await mount.cmd_enableCameraCableWrapFollowing.set_start()

In [ ]:
await mtcs.enable_ccw_following()

In [ ]:
def printLogMessage(data):
    print(f"{data.level}: {data.message}")
rot.evt_logMessage.callback = printLogMessage
await rot.cmd_setLogLevel.set_start(level=10, timeout=5)

In [ ]:
await camhex.cmd_clearError.set_start()

In [ ]:
await checkAOSCompStates(m1m3,m2,camhex,m2hex)

In [ ]:
await salobj.set_summary_state(camhex, state=salobj.State.ENABLED)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
await mtcs.enable_ccw_following()

In [ ]:
a =await mount.evt_logMessage.aget()
a.message, pd.to_datetime(a.private_sndStamp, unit='s')

In [ ]:
a = await rot.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
from lsst.ts.idl.enums.MTRotator import ApplicationStatus
a = await rot.evt_controllerState.aget()
print(a.applicationStatus, pd.to_datetime(a.private_sndStamp, unit='s'))

In [ ]:
ApplicationStatus(a.applicationStatus)

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
a = await m1m3.evt_errorCode.aget()
print(a.errorReport, pd.to_datetime(a.private_sndStamp, unit='s'))

#### Plot the above process

In [ ]:
#end = Time(datetime.now(), scale='tai')
end = Time('2021-07-13T19:18:00', scale='tai')
start = end - timedelta(seconds=1000)

In [ ]:
dfm2

In [ ]:
dfm = await client.select_time_series('lsst.sal.MTMount.elevation', '*', start, end, csc_index)
dfm1m3 = await client.select_time_series('lsst.sal.MTM1M3.logevent_appliedElevationForces', '*', start, end, csc_index)
dfm2 = await client.select_time_series('lsst.sal.MTM2.axialForce', '*', start, end, csc_index)
dfh = await client.select_time_series('lsst.sal.MTHexapod.application', '*', start, end, csc_index)

idx1=dfh.MTHexapodID==1
dfh1 = dfh[idx1]
idx2=dfh.MTHexapodID==2
dfh2 = dfh[idx2]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
plt.plot((dfm.actualPosition-dfm.actualPosition[0])*100, '--', label='mount elevation x 100 (deg)')
plt.plot(dfm1m3.yForces0-dfm1m3.yForces0[0], label='M1M3 elevation y-force 101 (N)')
plt.plot((dfm2.lutGravity0-dfm2.lutGravity0[0])*10, label='M2 elevation force B1 x 10 (N)')
plt.plot(dfh1.position1-dfh1.position1[0], label='Camera hexapod y (um)')
plt.plot(dfh2.position1-dfh2.position1[0], label='M2 hexapod y (um)')
plt.grid()
plt.legend()
plt.title('Changes in telemetry')

#### Check if the telescope is in tracking mode. If yes, need to stop stacking.

The alternative is to check "MT Mount status" dash board on Chronograf. Make sure there are three "False".

In [ ]:
mountStatus = await mount.evt_axesInPosition.aget(timeout=5.)
rotStatus = await rot.evt_inPosition.aget(timeout=5.)
trackingStatus = mountStatus.elevation and mountStatus.azimuth and rotStatus.inPosition
print('Are we tracking?', trackingStatus)

In [ ]:
await mtcs.stop_tracking()

### put mount elevation back to 90 deg, so that we can lower M1M3

In [ ]:
aa = await mount.tel_elevation.next(flush=True, timeout=5.)
current_el = aa.actualPosition

In [ ]:
await moveMountConstantV(mount,current_el, 90)

In [ ]:
await lowerM1M3(m1m3)

#### Close up. Put all simulators to standby

In [40]:
await mtcs.stop_tracking()

In [41]:
await salobj.set_summary_state(ptg, salobj.State.STANDBY)

[<State.FAULT: 3>, <State.STANDBY: 5>]

In [42]:
await salobj.set_summary_state(rot, salobj.State.OFFLINE)

RuntimeError: Error on cmd=cmd_disable, initial_state=2: msg='Command failed', ackcmd=(ackcmd private_seqNum=480200103, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Controller is not connected')

In [44]:
await salobj.set_summary_state(camhex, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [45]:
await salobj.set_summary_state(m2hex, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [46]:
await salobj.set_summary_state(m2, salobj.State.DISABLED)

RuntimeError: Error on cmd=cmd_standby, initial_state=2: msg='Command failed', ackcmd=(ackcmd private_seqNum=1839302522, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Complete : Failed')

In [47]:
await salobj.set_summary_state(m2, salobj.State.STANDBY)

[<State.DISABLED: 1>, <State.STANDBY: 5>]

In [48]:
await salobj.set_summary_state(m2, salobj.State.OFFLINE)

[<State.STANDBY: 5>, <State.OFFLINE: 4>]

In [ ]:
await lowerM1M3(m1m3)

m1m3 state DetailedState.LOWERING 2021-07-20 17:58:16.603898880


In [54]:
await salobj.set_summary_state(m1m3, salobj.State.STANDBY)

[<State.ENABLED: 2>,
 <State.DISABLED: 1>,
 <State.STANDBY: 5>,
 <State.OFFLINE: 4>]

In [55]:
await salobj.set_summary_state(mount, salobj.State.STANDBY)

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [ ]:
await rot.cmd_clearError.set_start()

In [ ]:
await m2.cmd_exitControl.set_start()

In [ ]:
await checkAOSCompStates(m1m3,m2,camhex, m2hex)

In [ ]:
await checkSlewCompStates(ptg,mount, rot)